# Objective
The objective of this project is to plan a proper portfolio that can minimize the loss of the asset by investing the stocks and etfs.

# [Link to FMP apis](https://site.financialmodelingprep.com/developer/docs)

In [180]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime, timezone, timedelta
import pytz

# Setting an API Key

In [181]:
api_key = "lDg6Ns4sehr6zVXCVAW2PWApFzdGZkdf"

# Setting the endpoints

In [182]:
ENDPOINTS = {
    'STOCK_LIST': 'https://financialmodelingprep.com/api/v3/stock/list',
    'COMPANY_PROFILE': 'https://financialmodelingprep.com/api/v3/profile',
    'HISTORICAL_DIVIDEND_YIELD': 'https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend',
    'STOCK_PRICES': 'https://wts-info-api.tossinvest.com/api/v2/stock-prices',
    'FOREIGN_EXCHANGE_PRICE': 'https://financialmodelingprep.com/api/v3/fx'
}

def get_request_endpoint(endpoint_type='STOCK_LIST' or 'COMPANY_PROFILE' or 'HISTORICAL_DIVIDEND_YIELD' or 'STOCK_PRICES', **kwargs):
    endpoint: str = ENDPOINTS[endpoint_type]

    if endpoint_type == 'COMPANY_PROFILE' or endpoint_type == 'HISTORICAL_DIVIDEND_YIELD':
        symbol: str = kwargs['symbol']
        endpoint = endpoint + f'/{symbol}'

    if endpoint_type == 'STOCK_PRICES': 
        stock_code: str = kwargs['stock_code']
        count: number = kwargs['count']
        to: str = kwargs['to']
        endpoint = endpoint + f'/{stock_code}/period-candles/day:1?count={count}&to={to}T05:00:00Z'
        return endpoint

    if endpoint_type == 'FOREIGN_EXCHANGE_PRICE':
        currency_pair = kwargs['currency_pair']
        endpoint = endpoint + f'/{currency_pair}'


    return endpoint + f'?apikey={api_key}'

# Retrieve the data

## Stock List

In [183]:
sl_res = requests.get(get_request_endpoint('STOCK_LIST'))

In [184]:
sl = sl_res.json()

In [185]:
sl_df = pd.DataFrame(sl)

In [186]:
sl_df.head()

,symbol,name,price,exchange,exchangeShortName,type
0,PMGOLD.AX,Perth Mint Gold,17.94,Australian Securities Exchange,ASX,etf
1,NECCLTD.NS,North Eastern Carrying Corporation Limited,33.83,National Stock Exchange of India,NSE,stock
2,SRF.NS,SRF Limited,2239.20,National Stock Exchange of India,NSE,stock
3,NOSA.ST,Nosa Plugs AB,0.65,Stockholm Stock Exchange,STO,stock
4,ALLCARGO.NS,Allcargo Logistics Limited,55.66,National Stock Exchange of India,NSE,stock


### Filter stock list

In [187]:
def filter_stocks(df: pd.DataFrame, stock_symbols: list):
    rows: list = []
    
    for symbol in stock_symbols:
        rows.append(df[df['symbol'] == symbol])

    filtered_df = pd.concat(rows)

    return filtered_df

In [219]:
sl_stocks = sl_df[sl_df['type'] == 'stock']
sl_etfs = sl_df[sl_df['type'] == 'etf']

In [220]:
target_stock_symbols = ['NVDA', 'VZ', 'MO', 'ENB', 'GOOGL', 'META', 'AAPL', 'BNS', 'IBM', 'KO']
target_etf_symbols = ['GLD', 'IAU']

In [223]:
filtered_sl_stocks = filter_stocks(sl_stocks, target_stock_symbols)
filtered_sl_etfs = filter_stocks(sl_etfs, target_etf_symbols)

In [224]:
filtered_sl_stocks

,symbol,name,price,exchange,exchangeShortName,type
37531,NVDA,NVIDIA Corporation,135.40,NASDAQ Global Select,NASDAQ,stock
36577,VZ,Verizon Communications Inc.,41.36,New York Stock Exchange,NYSE,stock
34685,MO,"Altria Group, Inc.",53.87,New York Stock Exchange,NYSE,stock
36034,ENB,Enbridge Inc.,40.42,New York Stock Exchange,NYSE,stock
37503,GOOGL,Alphabet Inc.,171.29,NASDAQ Global Select,NASDAQ,stock
38265,META,"Meta Platforms, Inc.",567.16,NASDAQ Global Select,NASDAQ,stock
37666,AAPL,Apple Inc.,222.91,NASDAQ Global Select,NASDAQ,stock
35123,BNS,The Bank of Nova Scotia,51.90,New York Stock Exchange,NYSE,stock
34657,IBM,International Business Machines Corporation,208.25,New York Stock Exchange,NYSE,stock
34633,KO,The Coca-Cola Company,65.01,New York Stock Exchange,NYSE,stock


In [225]:
filtered_sl_etfs

,symbol,name,price,exchange,exchangeShortName,type
43496,GLD,SPDR Gold Shares,252.47,New York Stock Exchange Arca,AMEX,etf
42383,IAU,iShares Gold Trust,51.62,New York Stock Exchange Arca,AMEX,etf


In [226]:
filtered_invest_cat = pd.concat([filtered_sl_stocks, filtered_sl_etfs])

In [227]:
filtered_invest_cat

,symbol,name,price,exchange,exchangeShortName,type
37531,NVDA,NVIDIA Corporation,135.40,NASDAQ Global Select,NASDAQ,stock
36577,VZ,Verizon Communications Inc.,41.36,New York Stock Exchange,NYSE,stock
34685,MO,"Altria Group, Inc.",53.87,New York Stock Exchange,NYSE,stock
36034,ENB,Enbridge Inc.,40.42,New York Stock Exchange,NYSE,stock
37503,GOOGL,Alphabet Inc.,171.29,NASDAQ Global Select,NASDAQ,stock
38265,META,"Meta Platforms, Inc.",567.16,NASDAQ Global Select,NASDAQ,stock
37666,AAPL,Apple Inc.,222.91,NASDAQ Global Select,NASDAQ,stock
35123,BNS,The Bank of Nova Scotia,51.90,New York Stock Exchange,NYSE,stock
34657,IBM,International Business Machines Corporation,208.25,New York Stock Exchange,NYSE,stock
34633,KO,The Coca-Cola Company,65.01,New York Stock Exchange,NYSE,stock


## Company profiles

In [192]:
def retrieve_company_profiles(df: pd.DataFrame):
    profiles = []
    
    for i, row in df.iterrows():
        endpoint: str = get_request_endpoint('COMPANY_PROFILE', symbol=row['symbol'])

        res = requests.get(endpoint)
        body = res.json()
        profiles.append(body[0])

    return profiles

In [228]:
company_profiles = retrieve_company_profiles(filtered_invest_cat)

In [229]:
cp_df = pd.DataFrame(company_profiles)

In [230]:
cp_df

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,NVDA,135.40,1.669,304393465,3321362000000,0.04,43.723-144.42,2.64,NVIDIA Corporation,USD,...,95051,77.28211,58.117890,https://images.financialmodelingprep.com/symbo...,1999-01-22,False,False,True,False,False
1,VZ,41.36,0.411,17949173,174110296800,2.71,35.4-45.36,-0.77,Verizon Communications Inc.,USD,...,10036,-20.87270,62.232696,https://images.financialmodelingprep.com/symbo...,1983-11-21,False,False,True,False,False
2,MO,53.87,0.684,7993992,91914071400,4.08,39.25-54.95,-0.59,"Altria Group, Inc.",USD,...,23230,-13.73173,67.601727,https://images.financialmodelingprep.com/symbo...,1985-07-01,False,False,True,False,False
3,ENB,40.42,0.904,3587589,88011316400,2.63,32.76-42.16,0.02,Enbridge Inc.,USD,...,T2P 3L8,32.95084,7.469163,https://images.financialmodelingprep.com/symbo...,1984-03-15,False,False,True,False,False
4,GOOGL,171.29,1.038,25438562,2103233936702,0.80,127.86-191.75,0.18,Alphabet Inc.,USD,...,94043,-29.85818,201.148181,https://images.financialmodelingprep.com/symbo...,2004-08-19,False,False,True,False,False
5,META,567.16,1.216,12383659,1434810807811,2.00,311.02-602.95,-0.42,"Meta Platforms, Inc.",USD,...,94025,102.09710,465.062896,https://images.financialmodelingprep.com/symbo...,2012-05-18,False,False,True,False,False
6,AAPL,222.91,1.239,50353639,3389145931000,1.00,164.08-237.49,-3.00,Apple Inc.,USD,...,95014,71.71229,151.197714,https://images.financialmodelingprep.com/symbo...,1980-12-12,False,False,True,False,False
7,BNS,51.90,0.974,1649451,63814683000,3.05,41.8-55.12,0.41,The Bank of Nova Scotia,USD,...,B3J 1W1,100.11185,-48.211851,https://images.financialmodelingprep.com/symbo...,2002-06-07,False,False,True,False,False
8,IBM,208.25,0.697,3913042,191829071000,6.68,145.28-237.37,1.53,International Business Machines Corporation,USD,...,10504,-45.17045,253.420450,https://images.financialmodelingprep.com/symbo...,1915-09-24,False,False,True,False,False
9,KO,65.01,0.608,14008671,280050078000,1.94,56.06-73.53,-0.30,The Coca-Cola Company,USD,...,30313,-38.20552,103.215525,https://images.financialmodelingprep.com/symbo...,1919-09-05,False,False,True,False,False


## Historical dividend yields

In [231]:
def retrieve_historical_dividiend_yields(df: pd.DataFrame) -> list:
    yield_data = []
    
    for i, row in df.iterrows():
        endpoint: str = get_request_endpoint('HISTORICAL_DIVIDEND_YIELD', symbol=row['symbol'])

        res = requests.get(endpoint)
        body = res.json()
        yield_data.append(body)

    return yield_data

In [232]:
historical_dividend_yields = retrieve_historical_dividiend_yields(filtered_sl_stocks)

In [233]:
def preprocess_historical_dividend_yields(l: list) -> dict:
    hdy_dict: dict = {}
    
    for e in l:
        hdy_dict[e['symbol']] = e['historical']

    return hdy_dict        

In [234]:
hdys = preprocess_historical_dividend_yields(historical_dividend_yields)

In [235]:
def flatten_data(d: dict):
    flattened_data = []

    for pk, child_dicts in d.items():
        for cd in child_dicts:
            record = {'stock_symbol': pk}
            record.update(cd)
            flattened_data.append(record)

    return pd.DataFrame(flattened_data)

In [236]:
hdys_df = flatten_data(hdys)
hdys_df = pd.DataFrame(hdys_df)

In [237]:
hdys_df.head()

,stock_symbol,date,label,adjDividend,dividend,recordDate,paymentDate,declarationDate
0,NVDA,2024-09-12,"September 12, 24",0.010,0.01,2024-09-12,2024-10-03,2024-08-28
1,NVDA,2024-06-11,"June 11, 24",0.010,0.01,2024-06-11,2024-06-28,2024-05-22
2,NVDA,2024-03-05,"March 05, 24",0.004,0.40,2024-03-06,2024-03-27,2024-02-21
3,NVDA,2023-12-05,"December 05, 23",0.004,0.40,2023-12-06,2023-12-28,2023-11-21
4,NVDA,2023-09-06,"September 06, 23",0.004,0.40,2023-09-07,2023-09-28,2023-08-23


## Stock Prices(Toss)

In [238]:
TOSS_STOCK_CODE = {
    'ENB': 'US20011030001',
    'NVDA': 'US19990122001',
    'GOOGL': 'US20040819002',
    'META': 'US20120518001',
    'AAPL': 'US19801212001',
    'MO': 'US19850702002',
    'BNS': 'US19690127001',
    'VZ': 'US19831121001',
    'IBM': 'US19490128001',
    'KO': 'US19240926001',
    'GLD': 'US20041118001',
    'IAU':, 'US20050128001'
}

STOCK_CODE_TO_SYMBOL = dict(zip(TOSS_STOCK_CODE.values(), TOSS_STOCK_CODE.keys()))

SyntaxError: expression expected after dictionary key and ':' (3552351824.py, line 13)

In [ ]:
def retrieve_historical_stock_prices_last_year(stock_code: str, today: str):
    endpoint: str = get_request_endpoint(endpoint_type='STOCK_PRICES', stock_code=stock_code, count=365, to=today)
    res = requests.get(endpoint)

    if res.status_code == 200:
        body = res.json()
        if body['result']['candles']:
            return res.json()
        else:
            raise RuntimeError(f'No more data to load - endpoint: {endpoint}')
    else:
        raise RuntimeError(f'Request failed - endpoint: {endpoint}')

In [ ]:
def retrieve_all_historical_stock_prices(stock_code: str):
    stock_prices = []
    more_data_exists = True
    tick = 0

    while more_data_exists:
        try:
            days = 365 * tick
            print(f"retrieving stock_prices of {stock_code} - {tick}")
            now_utc = datetime.now(timezone.utc)
            
            if days != 0:
                 now_utc = now_utc - timedelta(days=days)
                
            tz_kr = pytz.timezone('Asia/Seoul')
            now_kr = now_utc.astimezone(tz_kr)
            
            date_string = now_kr.strftime('%Y-%m-%d')
            stock_prices_last_year = retrieve_historical_stock_prices_last_year(stock_code, date_string)
            stock_prices.append(stock_prices_last_year)

            tick+=1
        except Exception as e:
            more_data_exists = False    
            print(f"retrieving stock_prices of {stock_code} - {tick} failed due to {e}")
            return stock_prices

In [ ]:
def retrieve_stock_prices(stock_codes: dict):
    stock_prices: dict = {}
    
    for stock_code in stock_codes:
        data = retrieve_all_historical_stock_prices(stock_codes[stock_code])
        stock_prices[stock_code] = data

    return stock_prices

In [ ]:
sp = retrieve_stock_prices(TOSS_STOCK_CODE)

In [ ]:
# input format : [{"results": {"code": str, "candles": list of dict}}]
# output format: {code : list of dicts}
def combine_results_of_stock(stock_prices_of_corp: list):
    candles = []
    
    for spc in stock_prices_of_corp:
        candles.extend(spc['result']['candles'])

    return candles

In [ ]:
# input format : {code : list of dicts}
# output format: {symbol : list of dicts}
def combine_results_of_stocks(stock_prices: dict):
    combined_prices: dict = {}
    
    for ck in stock_prices:
        print(ck)
        combined_prices[ck] = combine_results_of_stock(stock_prices[ck])

    return combined_prices    

In [ ]:
comb_sp = combine_results_of_stocks(sp)

In [ ]:
comb_sp_df = flatten_data(comb_sp)

## Foreign Currency Exchange(FOREX) price

The following is the list of description of each attributes:
- `ticker`: The currency pair being quoted. i.e. `USDKRW` means the unit is 1 USD to KRW(= KRW per USD)
- `bid`: The current bid price, which is the highest price of a buyer willing to pay for a unit of the base currency.
- `ask`: The ccurent ask price, which is the lowest price of a seller willing to sell for a unit of the base currency.
- `open`: The opening price of the currency pair for the current trading session.
- `low`: The lowest exchange rate recoreded during the current trading session.
- `high`: The highest exchange rate recorded during the current trading session.
- `changes`: The net change in the exchange rate from the open price to the current price.
- `date`: The timestamp of the exchange rate information was last updated.

In [ ]:
endpoint = get_request_endpoint('FOREIGN_EXCHANGE_PRICE', currency_pair='USDKRW')

In [ ]:
current_krw_per_usd = requests.get(endpoint).json()[0]

In [ ]:
current_krw_per_usd

In [ ]:
sl_df.columns

In [216]:
cp_df.columns

Index(['symbol', 'price', 'beta', 'volAvg', 'mktCap', 'lastDiv', 'range',
       'changes', 'companyName', 'currency', 'cik', 'isin', 'cusip',
       'exchange', 'exchangeShortName', 'industry', 'website', 'description',
       'ceo', 'sector', 'country', 'fullTimeEmployees', 'phone', 'address',
       'city', 'state', 'zip', 'dcfDiff', 'dcf', 'image', 'ipoDate',
       'defaultImage', 'isEtf', 'isActivelyTrading', 'isAdr', 'isFund'],
      dtype='object')

In [217]:
hdys_df.columns

Index(['stock_symbol', 'date', 'label', 'adjDividend', 'dividend',
       'recordDate', 'paymentDate', 'declarationDate'],
      dtype='object')

In [218]:
comb_sp_df.columns

Index(['stock_symbol', 'dt', 'open', 'high', 'low', 'close', 'volume',
       'amount', 'base', 'openKrw', 'highKrw', 'lowKrw', 'closeKrw', 'baseKrw',
       'openKrwDecimal', 'highKrwDecimal', 'lowKrwDecimal', 'closeKrwDecimal',
       'baseKrwDecimal'],
      dtype='object')